In [1]:
import os
import numpy as np
import pywt
import random
import seaborn as sns
import tensorflow as tf
from keras.layers import Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (
  Input, Conv1D, BatchNormalization, Activation, GlobalAveragePooling1D, 
  Dense, Dropout, GRU, Concatenate, LayerNormalization, MultiHeadAttention, 
  Reshape, Multiply, Softmax
)
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from utils import (
  encode_labels, check_gpu_availability, plot_loss_accuracytupian, 
  evaluate_model, plot_confusion_matrixtupian, plot_tsne, 
  plot_precision_recall_curve_multiclasstupian, plot_roc_curve_multiclasstupian, 
  AdjustLearningRateCallback, denoise2,count_labels,denoise2_iterative2,AdjustLearningRateCallback
)
from utils import plot_precision_recall_curve_multiclass,plot_roc_curve_multiclass2,calculate_g_mean,plot_confusion_matrix,plot_confusion_matrix2,plot_loss_accuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1DTranspose


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
check_gpu_availability()

GPU 可用


In [3]:
datafilename1 = "C:\\Users\\Administrator\\Desktop\\database\\cinc2017denoise.npz"
data1 = np.load(datafilename1, allow_pickle=True)
X_train, y_train, X_val, y_val, X_test, y_test = data1['ecgstrain'], data1['labelstrain'], data1['ecgsval'], data1['labelsval'], data1['ecgstest'], data1['labelstest']

In [4]:
y_train = encode_labels(y_train)
y_test = encode_labels(y_test)
y_val= encode_labels(y_val)
y_train = to_categorical(y_train, num_classes=4)
y_val=to_categorical(y_val, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dropout, Dense,MaxPool1D
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size, strides):
    shortcut = x
    # 主卷积路径
    x = Conv1D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # 如果shortcut的形状与x不匹配，则调整shortcut
    if shortcut.shape[-1] != x.shape[-1] or shortcut.shape[-2] != x.shape[-2]:
        shortcut = Conv1D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # 添加跳过连接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 19, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = residual_block(x, 32, 3, 2)
    x = residual_block(x, 64, 3, 2)
    x = residual_block(x, 128, 3, 2)
    x1 = GRU(128, return_sequences=False)(x)
    x2 = GRU(128, return_sequences=False)(x)
    x=x1+x2
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Example usage:
model =resnet((4500, 1), 4)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4500, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 4500, 32)     640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 4500, 32)     128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 4500, 32)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [6]:
callback = AdjustLearningRateCallback(factor=0.1, patience=2, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, batch_size=256, epochs=80, validation_data=(X_val, y_val), callbacks=[callback,early_stopping])

Epoch 1/80
55/55 [==============================] - 31s 340ms/step - loss: 0.4170 - accuracy: 0.5834 - val_loss: 0.4386 - val_accuracy: 0.5755
Epoch 2/80
55/55 [==============================] - 17s 319ms/step - loss: 0.4037 - accuracy: 0.5954 - val_loss: 0.4157 - val_accuracy: 0.5831
Epoch 3/80
55/55 [==============================] - 17s 308ms/step - loss: 0.3946 - accuracy: 0.6026 - val_loss: 0.3971 - val_accuracy: 0.6005
Epoch 4/80
55/55 [==============================] - 17s 308ms/step - loss: 0.3778 - accuracy: 0.6340 - val_loss: 0.3846 - val_accuracy: 0.6300
Epoch 5/80
55/55 [==============================] - 17s 312ms/step - loss: 0.3537 - accuracy: 0.6643 - val_loss: 0.3421 - val_accuracy: 0.6665
Epoch 6/80
55/55 [==============================] - 17s 309ms/step - loss: 0.3187 - accuracy: 0.7076 - val_loss: 0.3250 - val_accuracy: 0.6943
Epoch 7/80
55/55 [==============================] - 17s 316ms/step - loss: 0.2916 - accuracy: 0.7417 - val_loss: 0.3073 - val_accuracy: 0.7210

In [7]:
import numpy as np

def add_low_frequency_noise_multidim(data, snr, frequency_range=(0, 5), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [8]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,0)

In [9]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.7261326919721017
Recall: 0.6023705343205131
F1 Score: 0.6475079168333124
Accuracy: 0.7546259050683829
Class 1 - Precision: 0.6325301204819277, Recall: 0.46255506607929514, F1 Score: 0.5343511450381679
Class 2 - Precision: 0.8309768637532133, Recall: 0.8614257161892072, F1 Score: 0.845927379784102
Class 3 - Precision: 0.6228419654714475, Recall: 0.6355013550135501, F1 Score: 0.6291079812206573
Class 4 - Precision: 0.8181818181818182, Recall: 0.45, F1 Score: 0.5806451612903226
Class 1 Accuracy: 0.9263877715205149
Class 2 Accuracy: 0.8105390185036203
Class 3 Accuracy: 0.7775543041029767
Class 4 Accuracy: 0.994770716009654


In [10]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,5)

In [11]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.8242641422800469
Recall: 0.7278880339140612
F1 Score: 0.7631779526889068
Accuracy: 0.832260659694288
Class 1 - Precision: 0.7383966244725738, Recall: 0.7709251101321586, F1 Score: 0.7543103448275862
Class 2 - Precision: 0.8686176836861769, Recall: 0.9293804130579614, F1 Score: 0.8979723205664628
Class 3 - Precision: 0.7733755942947702, Recall: 0.6612466124661247, F1 Score: 0.7129291453615779
Class 4 - Precision: 0.9166666666666666, Recall: 0.55, F1 Score: 0.6874999999999999
Class 1 Accuracy: 0.9541432019308126
Class 2 Accuracy: 0.8724859211584876
Class 3 Accuracy: 0.8419147224456959
Class 4 Accuracy: 0.99597747385358


In [12]:
import numpy as np

def add_high_frequency_noise_multidim(data, snr, frequency_range=(0, 200), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [13]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,0)

In [14]:
evaluate_model(model,X_test_noisy,y_test)

Precision: 0.7420129997154645
Recall: 0.6119514720807047
F1 Score: 0.6595299959210408
Accuracy: 0.7614641995172968
Class 1 - Precision: 0.6736111111111112, Recall: 0.42731277533039647, F1 Score: 0.522911051212938
Class 2 - Precision: 0.8361075544174136, Recall: 0.8700866089273818, F1 Score: 0.8527587332680379
Class 3 - Precision: 0.625, Recall: 0.6504065040650406, F1 Score: 0.6374501992031872
Class 4 - Precision: 0.8333333333333334, Recall: 0.5, F1 Score: 0.625
Class 1 Accuracy: 0.9288012872083669
Class 2 Accuracy: 0.8185840707964602
Class 3 Accuracy: 0.7803700724054706
Class 4 Accuracy: 0.995172968624296


In [15]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,5)

In [16]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.8359427795756856
Recall: 0.730261382538149
F1 Score: 0.7706418168792397
Accuracy: 0.8366854384553499
Class 1 - Precision: 0.7863636363636364, Recall: 0.762114537444934, F1 Score: 0.7740492170022372
Class 2 - Precision: 0.8696194635059263, Recall: 0.9287141905396402, F1 Score: 0.8981958762886597
Class 3 - Precision: 0.7711213517665131, Recall: 0.6802168021680217, F1 Score: 0.7228221742260621
Class 4 - Precision: 0.9166666666666666, Recall: 0.55, F1 Score: 0.6874999999999999
Class 1 Accuracy: 0.9593724859211585
Class 2 Accuracy: 0.8728881737731295
Class 3 Accuracy: 0.8451327433628318
Class 4 Accuracy: 0.99597747385358
